<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

## Load Data

In [ ]:
train_csv = pd.read_csv(
    "data/autass_data2.csv",
    header=None,
    dtype=np.double,
)
data = np.array(train_csv.values[:, 1:50])
del train_csv

In [ ]:
X = data[:, 0:48]
y = data[:, 48].astype(int) - 1

yt = copy.copy(y)
yt[yt == 0] = 20
yt[yt == 1] = 21
yt[yt == 2] = 22
yt[yt == 3] = 23
yt[yt == 4] = 26
yt[yt == 5] = 24
yt[yt == 6] = 27
yt[yt == 7] = 29
yt[yt == 8] = 30
yt[yt == 9] = 25
yt[yt == 10] = 28
yt -= 20
y = yt
del yt

## Config

In [ ]:
epochs = 200
batch_size = 538
lr = 1

## Single Layer

### MLMVN [48-10-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-10-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 10)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(10, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-10-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-10-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=24f4db1bc3534991ae833c133d9f8d2d
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/24f4db1bc3534991ae833c133d9f8d2d/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-10-11]',
 'loss': 'ComplexMSE_adjusted_error'}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_3354/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 20:34:31,569 - clearml.frameworks - INFO - Found existing registered model id=caa96da5a415490ca1ea0f95b383f403 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-10-11.pt] reusing it.
Epoch 9 loss is 0.24948623262964803
Epoch 19 loss is 0.255540866164878
Epoch 29 loss is 0.21773538117269958
Epoch 39 loss is 0.21244534374484728
Epoch 49 loss is 0.2473424009802358
Epoch 59 loss is 0.20695948558149752
Epoch 69 loss is 0.29476683270052434
Epoch 79 loss is 0.24103331229757735
Epoch 89 loss is 0.27724582003427484
Epoch 99 loss is 0.2970595447560823
Epoch 109 loss is 0.3243128357279459
Epoch 119 loss is 0.39155765239889734
Epoch 129 loss is 0.4004725978900191
Epoch 139 loss is 0.3923780522292057
Epoch 149 loss is 0.32839938368714444
Epoch 159 loss is 0.28364692218207543
Epoch 169 loss is 0.27329238707230713
Epoch 179 loss is 0.23872938813414282
Epoch 189 loss is 0.24975022028484206
Epoch 199 loss is 0.23256343403758803
Train Acc.:  0.8357475537324275
Val

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-20-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-20-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 20)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(20, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        # writer.add_histogram("distribution centers", x + n_iter, i)
        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-20-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-20-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=9dcf5016dc4b4a7498fe8d6d4aeccbc9
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/9dcf5016dc4b4a7498fe8d6d4aeccbc9/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-20-11]',
 'loss': 'ComplexMSE_adjusted_error'}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_3354/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 20:37:36,382 - clearml.frameworks - INFO - Found existing registered model id=c337b94a22444d809d449783726d8ee2 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-20-11.pt] reusing it.
Epoch 9 loss is 0.24476062344530095
Epoch 19 loss is 0.17524997121406452
Epoch 29 loss is 0.17326589625589803
Epoch 39 loss is 0.1250403376833135
Epoch 49 loss is 0.10869102016395354
Epoch 59 loss is 0.12357040169954497
Epoch 69 loss is 0.1093577461378208
Epoch 79 loss is 0.12643380921745745
Epoch 89 loss is 0.11328518219237213
Epoch 99 loss is 0.10813876603811017
Epoch 109 loss is 0.10728799886326053
Epoch 119 loss is 0.09921635073504151
Epoch 129 loss is 0.1086356377434932
Epoch 139 loss is 0.10239754285657386
Epoch 149 loss is 0.09024639101528273
Epoch 159 loss is 0.10786458972685774
Epoch 169 loss is 0.10565815323356467
Epoch 179 loss is 0.10199340886787205
Epoch 189 loss is 0.10471597717547888
Epoch 199 loss is 0.10269092835171191
Train Acc.:  0.9379994017860958

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-50-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-50-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 50)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(50, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        # writer.add_histogram("distribution centers", x + n_iter, i)
        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-50-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-50-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=6e46e1426f0545abadae1da64a699266
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/6e46e1426f0545abadae1da64a699266/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-50-11]',
 'loss': 'ComplexMSE_adjusted_error'}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_3354/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 20:43:58,494 - clearml.frameworks - INFO - Found existing registered model id=bb96e63090904339bf87c4852d30bdb6 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-50-11.pt] reusing it.
Epoch 9 loss is 0.23981090673183392
Epoch 19 loss is 0.1350937123107344
Epoch 29 loss is 0.10171720961260702
Epoch 39 loss is 0.10397829965296723
Epoch 49 loss is 0.09251700067492151
Epoch 59 loss is 0.07677589171885311
Epoch 69 loss is 0.07428011077625409
Epoch 79 loss is 0.07424593369765878
Epoch 89 loss is 0.06974386731484813
Epoch 99 loss is 0.0625939765088816
Epoch 109 loss is 0.06981443338641498
Epoch 119 loss is 0.06748007620098263
Epoch 129 loss is 0.05657005163013897
Epoch 139 loss is 0.06359593600342729
Epoch 149 loss is 0.05271322280799125
Epoch 159 loss is 0.046950683837239236
Epoch 169 loss is 0.045860277729382115
Epoch 179 loss is 0.041940594701453865
Epoch 189 loss is 0.042831541937416555
Epoch 199 loss is 0.04799205469350261
Train Acc.:  0.97696876468

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-100-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-100-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 100)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(100, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        # writer.add_histogram("distribution centers", x + n_iter, i)
        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-100-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-100-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=e2d5d30ff2454b1db42f1bd3e874b935
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/e2d5d30ff2454b1db42f1bd3e874b935/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-100-11]',
 'loss': 'ComplexMSE_adjusted_error'}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_3354/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 20:48:36,824 - clearml.frameworks - INFO - Found existing registered model id=0f73e6db01fc42988672e4f44c0add5f [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-100-11.pt] reusing it.
Epoch 9 loss is 0.19675702136134435
Epoch 19 loss is 0.12691860293194868
Epoch 29 loss is 0.09963694001283371
Epoch 39 loss is 0.08266232549928246
Epoch 49 loss is 0.07721942842928552
Epoch 59 loss is 0.07040153803166502
Epoch 69 loss is 0.06772988802277076
Epoch 79 loss is 0.06808096965213609
Epoch 89 loss is 0.06277160699643843
Epoch 99 loss is 0.06304586082936255
Epoch 109 loss is 0.05947888365928943
Epoch 119 loss is 0.056828984384148414
Epoch 129 loss is 0.05502314843135909
Epoch 139 loss is 0.04954769492662839
Epoch 149 loss is 0.04978222914593748
Epoch 159 loss is 0.05478295147543594
Epoch 169 loss is 0.04662054887355213
Epoch 179 loss is 0.05929304252375296
Epoch 189 loss is 0.05981274953089023
Epoch 199 loss is 0.05613814120733131
Train Acc.:  0.97491774558

In [ ]:
task.mark_completed()
task.close()

## Multi Layer

### MLMVN [48-10-10-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-10-10-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 10)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(10, 10)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(10, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-10-10-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-10-10-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=ab30cdc3b6dc4a99bd4878cab67ae651
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/ab30cdc3b6dc4a99bd4878cab67ae651/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-10-10-11]',
 'loss': 'ComplexMSE_adjusted_error'}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_3354/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 21:00:37,622 - clearml.frameworks - INFO - Found existing registered model id=410edb2915b24269b7d34f2e38593dff [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-10-10-11.pt] reusing it.
Epoch 9 loss is 0.5352860994082685
Epoch 19 loss is 0.8573220308220786
Epoch 29 loss is 0.8846553943619413
Epoch 39 loss is 0.9282479485173039
Epoch 49 loss is 0.9304585998948428
Epoch 59 loss is 0.8896554528724867
Epoch 69 loss is 0.9020712551731598
Epoch 79 loss is 0.8780626206812697
Epoch 89 loss is 0.9192603465644281
Epoch 99 loss is 0.9080221700448701
Epoch 109 loss is 0.8979155400653883
Epoch 119 loss is 0.9165361645534543
Epoch 129 loss is 0.8875104726098959
Epoch 139 loss is 0.9474128621314876
Epoch 149 loss is 0.8798036890266178
Epoch 159 loss is 0.870499564750111
Epoch 169 loss is 0.8812218883029092
Epoch 179 loss is 0.8765249923026144
Epoch 189 loss is 0.8565456302997135
Epoch 199 loss is 0.8486741464838758
Train Acc.:  0.5802033927274282
Val Acc.:  0.5

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-20-20-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-20-20-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 20)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(20, 20)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(20, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        # writer.add_histogram("distribution centers", x + n_iter, i)
        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-20-20-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-20-20-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=cde3f58739e24fe2bf1c929abc8af866
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/cde3f58739e24fe2bf1c929abc8af866/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-20-20-11]',
 'loss': 'ComplexMSE_adjusted_error'}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_3354/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 21:07:48,901 - clearml.frameworks - INFO - Found existing registered model id=22ba5a4169ed406a9e74f40200bd29a1 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-20-20-11.pt] reusing it.
Epoch 9 loss is 0.33310382354735135
Epoch 19 loss is 0.3045317527345633
Epoch 29 loss is 0.26208047114462196
Epoch 39 loss is 0.5633030280355086
Epoch 49 loss is 0.6604212666659914
Epoch 59 loss is 0.6350045622908638
Epoch 69 loss is 0.6371642678164225
Epoch 79 loss is 0.6625240454923111
Epoch 89 loss is 0.618989009951792
Epoch 99 loss is 0.5788566858980941
Epoch 109 loss is 0.5580994790124282
Epoch 119 loss is 0.5665000001440373
Epoch 129 loss is 0.6303235022304811
Epoch 139 loss is 0.6471938510996637
Epoch 149 loss is 0.6645542731394183
Epoch 159 loss is 0.6454662272330662
Epoch 169 loss is 0.6336318409892531
Epoch 179 loss is 0.6200715265569947
Epoch 189 loss is 0.6395423856912091
Epoch 199 loss is 0.5969744437662491
Train Acc.:  0.7452036063752511
Val Acc.:  0

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-50-50-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-50-50-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 50)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(50, 50)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(50, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        # writer.add_histogram("distribution centers", x + n_iter, i)
        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-50-50-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-50-50-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=4910372b55c6431281234703cea5b873
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/4910372b55c6431281234703cea5b873/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-50-50-11]',
 'loss': 'ComplexMSE_adjusted_error'}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_3354/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 21:12:12,127 - clearml.frameworks - INFO - Found existing registered model id=f13061c5d03a4e96b788becd5e54443a [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-50-50-11.pt] reusing it.
Epoch 9 loss is 0.228445869053326
Epoch 19 loss is 0.1769025202901553
Epoch 29 loss is 0.13638551487260145
Epoch 39 loss is 0.12389236050933781
Epoch 49 loss is 0.10762237564656339
Epoch 59 loss is 0.09565055571155039
Epoch 69 loss is 0.08563549226626824
Epoch 79 loss is 0.07961991527154336
Epoch 89 loss is 0.08409453002137111
Epoch 99 loss is 0.11271795819004764
Epoch 109 loss is 0.09878662956475265
Epoch 119 loss is 0.0966812858882719
Epoch 129 loss is 0.08962629585333381
Epoch 139 loss is 0.08695483521719193
Epoch 149 loss is 0.08539091657127361
Epoch 159 loss is 0.09094220320688623
Epoch 169 loss is 0.09666773124329389
Epoch 179 loss is 0.0897644800428595
Epoch 189 loss is 0.09589186592403481
Epoch 199 loss is 0.09725851786233339
Train Acc.:  0.934452847925479

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-100-100-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-100-100-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 100)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(100, 100)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(100, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        # writer.add_histogram("distribution centers", x + n_iter, i)
        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

In [ ]:
model = Model(categories=categories, periodicity=periodicity)
criterion = ComplexMSE_adjusted_error.apply
optimizer = ECL(model.parameters(), lr=lr)

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-100-100-11]",
    tags=["mlmvn", "SDD", "single_run", "adjusted_loss"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": lr,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-100-100-11]",
    "loss": "ComplexMSE_adjusted_error",
}
task.connect(config_dict)

ClearML Task: created new task id=bae1f2339cae4e1880f988c0593eaa4c
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/bae1f2339cae4e1880f988c0593eaa4c/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-100-100-11]',
 'loss': 'ComplexMSE_adjusted_error'}

In [ ]:
x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

losses, scores = fit(
    model,
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    optimizer=optimizer,
    criterion=criterion,
    categories=categories,
    periodicity=periodicity,
)

model.load_state_dict(torch.load(PATH))

y_pred = model.predict(x_train)
acc = accuracy(y_pred.squeeze(), y_train)
print("Train Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Train Acc.",
    value=acc,
)

y_pred = model.predict(x_valid)
acc = accuracy(y_pred.squeeze(), y_valid)
print("Val Acc.: ", acc)
Logger.current_logger().report_single_value(
    name="Val Acc.",
    value=acc,
)
print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

/tmp/ipykernel_3354/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-22 21:18:41,784 - clearml.frameworks - INFO - Found existing registered model id=bbd65d869dea4025af46d264d3c7bdee [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-100-100-11.pt] reusing it.
Epoch 9 loss is 0.3850133302732534
Epoch 19 loss is 0.22928350502576889
Epoch 29 loss is 0.18473120489159367
Epoch 39 loss is 0.151906876717256
Epoch 49 loss is 0.13306868079012138
Epoch 59 loss is 0.12176301270741918
Epoch 69 loss is 0.11425308696387655
Epoch 79 loss is 0.1074521977072848
Epoch 89 loss is 0.09860157716737963
Epoch 99 loss is 0.09525496381491966
Epoch 109 loss is 0.09652682279150973
Epoch 119 loss is 0.09278257367639477
Epoch 129 loss is 0.09085598133404435
Epoch 139 loss is 0.0890178305214046
Epoch 149 loss is 0.08561603038010733
Epoch 159 loss is 0.08194544436834562
Epoch 169 loss is 0.07805339709847767
Epoch 179 loss is 0.08145146698926171
Epoch 189 loss is 0.07395142641579694
Epoch 199 loss is 0.07222291978897587
Train Acc.:  0.9455838995000

In [ ]:
task.mark_completed()
task.close()